In [52]:
import pandas as pd
import requests
import zipfile
from datetime import datetime
from dateutil.relativedelta import relativedelta
import quandl

pd.options.display.float_format = '{:.4f}'.format

# Código completo na descrição

In [55]:
anos = 2 #por conta da fonte de dados, só funciona até começo de 2021
mes = []
ano = []

for i in range(anos * 12 - 1):
    mes.append((datetime.today()-relativedelta(months=i+1)).month)
    ano.append((datetime.today()-relativedelta(months=i+1)).year)

mes = mes[::-1]
ano = ano[::-1]

for i in range(len(mes)):
    if len(str(mes[i])) < 2:
        mes[i] = "0" + str(mes[i])
    else:
        mes[i] = str(mes[i])

for i in range(len(ano)):
    ano[i] = str(ano[i])

print(mes)
print(ano)

['04', '05', '06', '07', '08', '09', '10', '11', '12', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '01', '02']
['2021', '2021', '2021', '2021', '2021', '2021', '2021', '2021', '2021', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2022', '2023', '2023']


In [56]:
dados_fundos = pd.DataFrame()

for i in range(len(mes)):
    url = f'https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{ano[i]}{mes[i]}.zip'

    download = requests.get(url)

    with open(f"inf_diario_fi_{ano[i]}{mes[i]}.zip", "wb") as arquivo_cvm:

        arquivo_cvm.write(download.content) 
        
    arquivo_zip = zipfile.ZipFile(f"inf_diario_fi_{ano[i]}{mes[i]}.zip")

    dados_fundos_mes = pd.read_csv(arquivo_zip.open(arquivo_zip.namelist()[0]), sep = ";", encoding = 'ISO-8859-1')

    dados_fundos = dados_fundos.append(dados_fundos_mes)

dados_fundos.shape

    

C:\Users\rmnf\AppData\Local\Temp\ipykernel_19396\2481153283.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados_fundos = dados_fundos.append(dados_fundos_mes)
C:\Users\rmnf\AppData\Local\Temp\ipykernel_19396\2481153283.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados_fundos = dados_fundos.append(dados_fundos_mes)
C:\Users\rmnf\AppData\Local\Temp\ipykernel_19396\2481153283.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados_fundos = dados_fundos.append(dados_fundos_mes)
C:\Users\rmnf\AppData\Local\Temp\ipykernel_19396\2481153283.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dados_fundos = dados_fundos.appen

(11043995, 9)

In [57]:
dados_fundos.tail()

,TP_FUNDO,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
444213,FI,97.929.213/0001-34,2023-02-22,67257749.8900,10.3160,78394219.6100,0.0000,0.0000,2
444214,FI,97.929.213/0001-34,2023-02-23,77462678.4600,10.3171,78403192.7300,0.0000,0.0000,2
444215,FI,97.929.213/0001-34,2023-02-24,78120590.3500,10.3164,78397536.7100,0.0000,0.0000,2
444216,FI,97.929.213/0001-34,2023-02-27,68690770.1200,10.3256,78467221.6000,0.0000,0.0000,2
444217,FI,97.929.213/0001-34,2023-02-28,78214503.2500,10.3286,78490459.4900,0.0000,0.0000,2


In [58]:
dados_cadastro = pd.read_csv('https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv', 
                             sep = ";", encoding = 'ISO-8859-1')

dados_cadastro = dados_cadastro[['CNPJ_FUNDO', 'DENOM_SOCIAL']]

dados_cadastro = dados_cadastro.drop_duplicates()

dados_cadastro

C:\Users\rmnf\AppData\Local\Temp\ipykernel_19396\3548788005.py:1: DtypeWarning: Columns (14,17,18,20,22,24,27,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  dados_cadastro = pd.read_csv('https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv',


,CNPJ_FUNDO,DENOM_SOCIAL
0,00.000.684/0001-21,DEUTSCHE BANK FDO APLIC QUOTAS FDO INV FINANCE...
1,00.000.731/0001-37,ITAMARITI CASH FUNDO APLICACAO QUOTAS FDOS INV...
2,00.000.732/0001-81,FUNDO APLIC. QUOTAS DE F.I. SANTANDER CURTO PRAZO
3,00.000.740/0001-28,FUNDO DE APLIC EM QUOTAS DE FUNDOS DE INV BMC ...
4,00.000.749/0001-39,BALANCE FUNDO APLICACAO QUOTAS FUNDO INVESTIME...
...,...,...
71355,28.850.140/0001-78,FUNDO DE FINANCIAMENTO DA INDUSTRIA CINEMATOGR...
71356,28.912.043/0001-62,HAPVIDA - FUNDO DE FINANCIAMENTO DA INDUSTRIA ...
71357,31.674.280/0001-47,MEDIA VENTURE ESG INCENTIVADO - FUNDO DE FINAN...
71358,32.222.962/0001-81,Fundo de Financiamento da Indústria Cinematogr...


In [59]:
data_inicio = (dados_fundos['DT_COMPTC'].sort_values(ascending = True).unique())[0]
data_fim = (dados_fundos['DT_COMPTC'].sort_values(ascending = True).unique())[-1]
        

dados_fundos_filtrado = dados_fundos[(dados_fundos['DT_COMPTC'].isin([data_inicio, data_fim]))]       

dados_fundos_filtrado.tail()

,TP_FUNDO,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
444145,FI,97.548.164/0001-90,2023-02-28,59551420.8400,0.1519,59751584.0700,0.0000,0.0000,1
444163,FI,97.548.167/0001-23,2023-02-28,5337680974.2100,4.4325,5126409391.8300,2852045.0000,0.0000,12
444181,FI,97.711.801/0001-05,2023-02-28,86573287.2100,2.8943,86545827.3600,0.0000,0.0000,1
444199,FI,97.929.197/0001-80,2023-02-28,132053514.5000,4.7015,132211376.8600,0.0000,0.0000,13
444217,FI,97.929.213/0001-34,2023-02-28,78214503.2500,10.3286,78490459.4900,0.0000,0.0000,2


In [60]:
dados_fundos_filtrado.head()

,TP_FUNDO,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
0,FI,00.017.024/0001-53,2021-04-01,1088904.8000,27.5044,1085588.5900,0.0000,0.0000,1
20,FI,00.068.305/0001-35,2021-04-01,42727667.1300,27.4595,42714160.3800,0.0000,68633.0400,7065
40,FI,00.071.477/0001-68,2021-04-01,32867494563.2000,10.0339,32865040339.3400,2228191664.4700,2520595488.9100,300503
60,FI,00.073.041/0001-08,2021-04-01,17244574.2300,28.8837,17238589.3200,0.0000,0.0000,933
80,FI,00.083.181/0001-67,2021-04-01,16867350010.0700,723.3712,16858070239.3100,0.0000,0.0000,4


In [61]:
dados_fundos_filtrado.shape

(44888, 9)

In [62]:
base_final = pd.merge(dados_fundos_filtrado, dados_cadastro, how = "left", 
                      left_on = ["CNPJ_FUNDO"], right_on = ["CNPJ_FUNDO"])

base_final = base_final[['CNPJ_FUNDO', 'DENOM_SOCIAL', 'DT_COMPTC', 'VL_QUOTA', 'VL_PATRIM_LIQ', 'NR_COTST']]

base_final

,CNPJ_FUNDO,DENOM_SOCIAL,DT_COMPTC,VL_QUOTA,VL_PATRIM_LIQ,NR_COTST
0,00.017.024/0001-53,FUNDO DE INVESTIMENTO RENDA FIXA EXPONENCIAL,2021-04-01,27.5044,1085588.5900,1
1,00.068.305/0001-35,FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE IN...,2021-04-01,27.4595,42714160.3800,7065
2,00.071.477/0001-68,BB RENDA FIXA AUTOMÁTICO EMPRESA SIMPLES FUNDO...,2021-04-01,10.0339,32865040339.3400,300503
3,00.073.041/0001-08,BB BESC RENDA FIXA PRÁTICO CRÉDITO PRIVADO FUN...,2021-04-01,28.8837,17238589.3200,933
4,00.083.181/0001-67,OPP I FUNDO DE INVESTIMENTO EM AÇÕES BDR NÍVEL...,2021-04-01,723.3712,16858070239.3100,4
...,...,...,...,...,...,...
44887,97.548.164/0001-90,DYNAMIC II - FUNDO DE INVESTIMENTO MULTIMERCAD...,2023-02-28,0.1519,59751584.0700,1
44888,97.548.167/0001-23,JGP STRATEGY MASTER FUNDO DE INVESTIMENTO MULT...,2023-02-28,4.4325,5126409391.8300,12
44889,97.711.801/0001-05,953 FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS D...,2023-02-28,2.8943,86545827.3600,1
44890,97.929.197/0001-80,LECT FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS ...,2023-02-28,4.7015,132211376.8600,13


In [92]:
fundos_prev = base_final[base_final['DENOM_SOCIAL'].str.contains("PREV", na = False)]
fundos_prev = fundos_prev[fundos_prev.VL_PATRIM_LIQ >= 10000000000]
fundos_prev = fundos_prev[fundos_prev.NR_COTST == 1]
fundos_prev = fundos_prev.sort_values(by="VL_PATRIM_LIQ", ascending=False)

fundos_prev

,CNPJ_FUNDO,DENOM_SOCIAL,DT_COMPTC,VL_QUOTA,VL_PATRIM_LIQ,NR_COTST
2151,07.919.956/0001-30,BRASILPREV RT FIX VI FUNDO DE INVESTIMENTO EM ...,2021-04-01,3.5275,36739582925.1800,1
569,03.537.407/0001-40,BRASILPREV RT FIX II FUNDO DE INVESTIMENTO EM ...,2021-04-01,8.8525,35567523225.6200,1
1622,06.001.785/0001-01,BRASILPREV RT FIX VII FUNDO DE INVESTIMENTO EM...,2021-04-01,3.2717,31873069661.8000,1
22220,07.919.956/0001-30,BRASILPREV RT FIX VI FUNDO DE INVESTIMENTO EM ...,2023-02-28,4.0647,31020291375.5500,1
20750,03.537.407/0001-40,BRASILPREV RT FIX II FUNDO DE INVESTIMENTO EM ...,2023-02-28,10.1520,28452408954.7400,1
31191,29.259.706/0001-54,ITAÚ FLEXPREV HIGH YIELD RENDA FIXA CRÉDITO PR...,2023-02-28,13.2600,27641687129.0500,1
21739,06.001.785/0001-01,BRASILPREV RT FIX VII FUNDO DE INVESTIMENTO EM...,2023-02-28,3.8023,26988069521.1800,1
1217,05.061.121/0001-67,BRASILPREV RT FIX C FUNDO DE INVESTIMENTO EM C...,2021-04-01,6.1017,17560628575.3500,1
10839,27.467.928/0001-37,ITAÚ FLEXPREV RENDA FIXA VISION FUNDO DE INVES...,2021-04-01,12.4219,16737561086.8800,1
17281,36.347.890/0001-50,CARTEIRA ITAÚ ASSET DE PREVIDÊNCIA FICFI MULTI...,2021-04-01,10.9965,14835507218.6500,1


# Patrimônio e performance do fundo em dezembro

In [93]:
maior_fundo = fundos_prev[(fundos_prev['CNPJ_FUNDO'] == fundos_prev["CNPJ_FUNDO"].iloc[0])]

maior_fundo = maior_fundo.sort_values(by="DT_COMPTC")

patrimonio_do_fundo =  "R$ " + str((maior_fundo['VL_PATRIM_LIQ'].iloc[-1]/1000000000).round(2)) + " Bi"

patrimonio_do_fundo

'R$ 31.02 Bi'

In [94]:
                                       
retorno = (maior_fundo['VL_QUOTA'].iloc[-1]/maior_fundo['VL_QUOTA'].iloc[0] - 1)* 100

retorno.round(2)

15.23

In [95]:
quandl.ApiConfig.api_key = "YUSDTrHyCjE7Gd8dP_yn"

ipca_mes = quandl.get("BCB/433", start_date = f"{ano[0]}-{mes[0]}-01")
cdi_mes = quandl.get("BCB/4391", start_date = f"{ano[0]}-{mes[0]}-01")

ipca_mes

,Value
Date,
2021-04-30,0.3100
2021-05-31,0.8300
2021-06-30,0.5300
2021-07-31,0.9600
2021-08-31,0.8700
2021-09-30,1.1600
2021-10-31,1.2500
2021-11-30,0.9500
2021-12-31,0.7300


In [96]:
cdi_mes['retorno_acumulado'] = (1 + cdi_mes['Value']/100).cumprod() - 1
ipca_mes['retorno_acumulado'] = (1 + ipca_mes['Value']/100).cumprod() - 1

cdi_mes

,Value,retorno_acumulado
Date,,
2021-04-30,0.2100,0.0021
2021-05-31,0.2700,0.0048
2021-06-30,0.3100,0.0079
2021-07-31,0.3600,0.0115
2021-08-31,0.4300,0.0159
2021-09-30,0.4400,0.0204
2021-10-31,0.4900,0.0254
2021-11-30,0.5900,0.0314
2021-12-31,0.7700,0.0394


In [97]:
retorno_cdi = ((cdi_mes['retorno_acumulado'].iloc[-1]) * 100)

retorno_ipca = ((ipca_mes['retorno_acumulado'].iloc[-1]) * 100)

print("Retorno fundo: " + str(retorno.round(2)) + " %")
print("Retorno CDI: " + str(retorno_cdi.round(2)) + " %")
print("Retorno IPCA: " + str(retorno_ipca.round(2)) + " %" + "\n")
print("Retorno %CDI: " + str((retorno/retorno_cdi * 100).round(2)) + " %")
print("Retorno IPCA: IPCA + " + str((retorno - retorno_ipca).round(2)) + " %")

Retorno fundo: 15.23 %
Retorno CDI: 19.32 %
Retorno IPCA: 14.69 %

Retorno %CDI: 78.82 %
Retorno IPCA: IPCA + 0.54 %


Maior fundo previdenciário em patrimônio, não ganha para o CDI e não bate uma renda fixa pós em inflação mesmo correndo mais risco, é bom escolher bem o fundo previdenciário